In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers
import games_configurations
env_name = 'RoboschoolHumanoidFlagrun-v1'#'RoboschoolHalfCheetah-v1'#'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#a2c_config = games_configurations.pendulum_lstm_config
#a2c_config = games_configurations.halfcheetah_lstm_config_v2
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
a2c_config = games_configurations.flagrun_lstm_config
#a2c_config = games_configurations.roboschoolhumanoid_config
#a2c_config = games_configurations.bipedalwalker_lstm_config
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-07-07 10:57:53,989	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-07_10-57-53_26723/logs.
2019-07-07 10:57:54,105	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:47872 to respond...
2019-07-07 10:57:54,254	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:52683 to respond...
2019-07-07 10:57:54,261	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-07-07 10:57:54,325	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-07_10-57-53_26723/logs.
2019-07-07 10:57:54,327	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:47872',
 'object_store_address': '/tmp/ray/session_2019-07-07_10-57-53_26723/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-07_10-57-53_26723/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks



agent = A2CAgent(sess,'run', obs_space, False, action_space, a2c_config)
#agent.restore('nn/hardBipedalWalker-v2')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


2019-07-07 10:57:55,477	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


0
1
2
3
4
5
6
7
0
Instructions for updating:
Use tf.cast instead.
(pid=26806) 
(pid=26806) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=26806) For more information, please see:
(pid=26806)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=26806)   * https://github.com/tensorflow/addons
(pid=26806) If you depend on functionality not listed there, please file an issue.
(pid=26806) 
(pid=26804) 
(pid=26804) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=26804) For more information, please see:
(pid=26804)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=26804)   * https://github.com/tensorflow/addons
(pid=26804) If you depend on functionality not listed there, please file an issue.
(pid=26804) 
(pid=26810) 
(pid=26810) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=26810) For more information

(pid=26844) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=26844)   result = entry_point.load(False)
(pid=26831) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=26831)   result = entry_point.load(False)
(pid=26843) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=26843)   result = entry_point.load(False)
(pid=26837) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=26837)   result = entry_po

Frames per seconds:  1925.9950048748617
Frames per seconds:  1918.94481626145
Frames per seconds:  1920.6866835866515
Frames per seconds:  1925.5369341503103
Frames per seconds:  1917.0247442299333
Frames per seconds:  1905.30064486822
Frames per seconds:  1920.4562520116108
Frames per seconds:  1921.0674757724767
Frames per seconds:  1931.498261174563
Frames per seconds:  1899.388491088533
Frames per seconds:  1924.70958107111
Frames per seconds:  1927.587750468051
Frames per seconds:  1874.7655464247946
Frames per seconds:  1924.0624215176326
Frames per seconds:  1923.9133169663205
Frames per seconds:  1914.0761214818103
Frames per seconds:  1909.5672084132304
Frames per seconds:  1922.9809144369956
Frames per seconds:  1915.522066692341
Frames per seconds:  1917.883719097333
Frames per seconds:  1928.505955135347
Frames per seconds:  1911.7046856678169
Frames per seconds:  1924.880052225374
Frames per seconds:  1905.2629807217913
Frames per seconds:  1917.6328755991776
Frames per se

Frames per seconds:  1903.2937379303544
Frames per seconds:  1906.7883273476593
Frames per seconds:  1899.1063005210478
Frames per seconds:  1877.6898887378297
Frames per seconds:  1917.6571703497248
Frames per seconds:  1892.2081523016677
Frames per seconds:  1905.7418964101034
Frames per seconds:  1911.01538109349
Frames per seconds:  1914.1485241822002
Frames per seconds:  1876.4181061636407
Frames per seconds:  1907.2130123096383
Frames per seconds:  1917.6564211631965
Frames per seconds:  1884.7097570400836
Frames per seconds:  1915.6131082121415
Frames per seconds:  1914.9829365695527
Frames per seconds:  1905.9487224887575
Frames per seconds:  1895.9057789452488
Frames per seconds:  1882.8934060225592
Frames per seconds:  1889.5510177572678
Frames per seconds:  1887.5603370452523
Frames per seconds:  1862.9587343365374
Frames per seconds:  1910.3683148657894
Frames per seconds:  1913.5180901095116
Frames per seconds:  1881.375181181615
Frames per seconds:  1901.3945317723592
Fra

Frames per seconds:  1896.081178055905
Frames per seconds:  1861.8628343934636
Frames per seconds:  1940.4976193284522
Frames per seconds:  1923.5269270068595
Frames per seconds:  1918.3995271633985
Frames per seconds:  1939.6378724968545
Frames per seconds:  1930.9210742364444
Frames per seconds:  1920.9137876593654
Frames per seconds:  1865.436574958533
Frames per seconds:  1925.760761468193
Frames per seconds:  1944.2895336247093
Frames per seconds:  1924.9573187045173
Frames per seconds:  1929.7112688526672
Frames per seconds:  1941.2397270503504
Frames per seconds:  1880.429411475293
Frames per seconds:  1717.7858945056955
Frames per seconds:  1939.1611420752806
Frames per seconds:  1901.9311428152482
Frames per seconds:  1916.7535419138924
Frames per seconds:  1932.2649578326075
Frames per seconds:  1926.8884052813296
Frames per seconds:  1917.0345307637288
Frames per seconds:  1945.9428901623257
Frames per seconds:  1924.320284716443
Frames per seconds:  1932.6826961117654
Frame

Frames per seconds:  1862.8501063543163
Frames per seconds:  1776.226912131102
Frames per seconds:  1813.9020213573808
Frames per seconds:  1583.9353421237652
Frames per seconds:  1773.6514161509072
Frames per seconds:  1779.2711554802254
Frames per seconds:  1767.6893946471735
saving next best rewards:  14.50828643474842
Frames per seconds:  1835.068902021208
Frames per seconds:  1855.5325353042595
Frames per seconds:  1867.3030527403819
Frames per seconds:  1760.6999136758538
Frames per seconds:  1740.8914794163854
Frames per seconds:  1933.202906859392
Frames per seconds:  1933.8509520306477
Frames per seconds:  1882.8394952592744
Frames per seconds:  1855.1109704949804
Frames per seconds:  1802.4798748914163
Frames per seconds:  1832.9734193624656
Frames per seconds:  1935.486868106043
Frames per seconds:  1938.6378842574154
Frames per seconds:  1925.6873697289336
Frames per seconds:  1904.594887781174
Frames per seconds:  1654.015033792155
Frames per seconds:  1772.6695643706182
F

In [ ]:
agent.save('nn/latest_run')
ray.shutdown()

In [ ]:
import gym
gym.envs.registry.all()